In [4]:
import pandas as pd
from chm import CascadeHierarquicalModel
from decomposers.hilbert_huang import EMDDecomposition
from regressors.lstm_regressor import LSTMRegressor
from data_retriever import DataRetriever
from pre_processing import PreProcessing
from scalers.min_max import MinMax
from scalers.scaler import Scaler
from scalers.standard import Standard


 

In [ ]:
dr = DataRetriever()
pp = PreProcessing()

yahoo_ds = dr.download_yahoo_data()
scaled_yahoo_ds = pp.values_scaler_partition_by_2(yahoo_ds, 'Ticker', 'Var', 'value', MinMax)


In [2]:
scaled_yahoo_ds

,Date,Var,Ticker,value,stock_market
0,2010-01-04,Open,ABEV3.SA,0.013577,B3
1,2010-01-04,High,ABEV3.SA,0.018107,B3
2,2010-01-04,Low,ABEV3.SA,0.012811,B3
3,2010-01-04,Close,ABEV3.SA,0.020651,B3
4,2010-01-04,Volume,ABEV3.SA,0.002845,B3
...,...,...,...,...,...
13630,2021-01-25,Volume,7011.T,0.095970,TSE
13631,2021-01-26,Volume,7011.T,0.103145,TSE
13632,2021-01-27,Volume,7011.T,0.086186,TSE
13633,2021-01-28,Volume,7011.T,0.262468,TSE


In [ ]:

sdate
edate
pd.date_range(sdate,edate-timedelta(days=1),freq='d')